---
  authors: 
    - name: Mathis Derenne
      affiliation: M2 MIASHS - Université de Lyon
      github: mathisdrn
    - name: Djida Boukari
      affiliation: M2 MIASHS - Université de Lyon
    - name: Ines Nakhli
      affiliation: M2 MIASHS - Université de Lyon
---

# Introduction

### 1. Contexte et objectifs
Le projet SigBERT propose une nouvelle approche de modélisation du risque patient en transformant des séries temporelles d'embeddings de textes cliniques (OncoBERT) en signatures mathématiques (Path Signatures). L'objectif de cette étude est de coupler cette représentation riche des données à des méthodes de Prédiction Conforme (Conformal Prediction). Il s'agit de dépasser la simple estimation ponctuelle du risque fournie par les modèles de survie classiques pour offrir des intervalles de prédiction valides avec des garanties statistiques de couverture, un prérequis essentiel pour l'aide à la décision clinique.

### 2 Description des données
Les données sont issues des fichiers `df_study_L18_w6.csv` et `df_study_L36_w6.csv`.

**Structure et Notation**

Soit un jeu de données $\mathcal{D} = \{(X_i, T_i, \delta_i)\}_{i=1}^n$ où chaque patient $i$ est défini par :
- **Covariables ($X_i$) :** Les coefficients issus des signatures de chemins (Signature features). Ces vecteurs de haute dimension capturent les interactions non-linéaires et l'ordre séquentiel des événements cliniques.
- **Temps observé ($T_i$) :** $T_i = \min(T^*_i, C_i)$, où $T^*_i$ est le temps réel jusqu'au décès et $C_i$ le temps de censure.
- **Indicateur de censure ($\delta_i$) :** $\delta_i = \mathbb{1}(T^*_i \leq C_i)$.
  - $\delta_i = 1$ : Décès observé (événement).
  - $\delta_i = 0$ : Censure (perdu de vue ou fin d'étude).

**Prétraitement**
- Données manquantes : les jeux de données n'ont pas de données manquantes, aucune imputation n'est nécessaire.
- Standardisation : Centrage et réduction ($Z$-score) appliqués aux covariables $X$.
- Partitionnement : Division en trois ensembles distincts : Entraînement ($\mathcal{D}_{train}$), Calibration ($\mathcal{D}_{cal}$) et Test ($\mathcal{D}_{test}$).

### 3. Approche méthodologique

**Modélisation du Risque (Cox Penalisé)**

Nous modélisons le risque instantané de décès $\lambda(t|X)$ via un modèle à risques proportionnels de Cox régularisé. La fonction de risque est définie par :$$\lambda(t|X) = \lambda_0(t) \exp(\beta^\top X)$$

Compte tenu de la haute dimensionnalité des signatures, nous estimons les coefficients $\beta$ en minimisant la moins log-vraisemblance partielle en testant différentes combinaisons de régularisations par **validation croisée**: sans régularisation, L1, L2 et ElasticNet (L1+L2)
$$\hat{\beta} = \arg\min_{\beta} \left[ -l(\beta) + \alpha \left( \rho \|\beta\|_1 + \frac{1-\rho}{2} \|\beta\|_2^2 \right) \right]$$

### 4. Prédiction conforme
L'objectif est de construire un ensemble de prédiction $\mathcal{C}(X)$ pour un horizon temporel $t^*$ (par ex. 3 ans) tel que : $$\mathbb{P}(Y \in \mathcal{C}(X)) \geq 1 - \alpha$$
Où $Y \in \{0, 1\}$ est le statut binaire du patient à $t^*$ (0: Décédé, 1: Vivant).

**Score de non-conformité**

Pour un patient $i$ de l'ensemble de calibration, connaissant sa vraie issue $y_i$ et ses covariables $x_i$, le score de non-conformité $s(x_i, y_i)$ est défini ainsi :$$s(x_i, y_i) = 
\begin{cases} 
1 - \hat{S}(t^*|x_i) & \text{si } y_i = 1 \text{ (Vivant)} \\
\hat{S}(t^*|x_i) & \text{si } y_i = 0 \text{ (Décédé)}
\end{cases}$$

**Gestion de la censure**

Le défi majeur réside dans le fait que pour les patients censurés avant $t^*$ ($T_i < t^*$ et $\delta_i=0$), le vrai label $Y_i$ est inconnu. Pour traiter cela, on envisage l'approche suivante:

**Censure de Type-I (Approche Heuristique)**

Cette méthode repose sur l'hypothèse que la censure est purement administrative et indépendante de l'état de santé du patient (MCAR). Nous effectuons une analyse sur les "cas complets" à l'horizon $t^*$.

- **Procédure :** Nous restreignons l'ensemble de calibration $\mathcal{D}_{cal}$ aux seuls patients dont le statut à $t^*$ est connu. On conserve le patient $i$ si :
    - Il est décédé avant l'horizon : $T_i \le t^*$ et $\delta_i = 1$.
    - Son suivi dépasse l'horizon : $T_i > t^*$.
- **Calibration :** On calcule le quantile empirique standard $\hat{q}_{1-\alpha}$ des scores de ce sous-ensemble réduit.
- **Avantage/Inconvénient :** Très simple à implémenter, mais entraîne une réduction de la taille de l'échantillon, rendant les intervalles potentiellement plus larges (moins efficaces) pour maintenir la couverture. D'autres approches plus avancées pourraient être explorées comme l'IPCW (Inverse Probability Censoring Weights) qui consiste à repondérer les observations en fonction de leur probabilité d'être censurées ou le calcul de scores de non-conformité adaptés à la survie (Right-Censored Score) (voir Candes et al.).

### 5. Critères d'évaluation

La performance sera jugée sur deux axes :
1. **Discrimination et calibration (modèle de base) :** C-index (Concordance de Harrell) et AUC dépendante du temps.
2. **Qualité des intervalles (prédiction conforme) :**
   - Couverture Marginale : Doit être statistiquement proche de $1-\alpha$.
   - Taille moyenne des ensembles de prédictions : plus les ensembles sont petits, plus ils sont informatifs.
   - Proportion de cas non informatifs (ensembles $\{décédé, vivant\}$).